In [ ]:
#Import Python Libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
#Load/transform data through sparksql
query = '''
select t1.EmployeeHashID, t1.date, t1.AssignedBuilding_SwipedIn, t1.assignedbuilding, t1.assignedbuilding_region, 
t1.assignedbuilding_country, t1.RoomTypeDesc, t1.HirePost20200301, t1.JobCategory, t2.EmployeeHashID as Mngr_ID, 
CASE WHEN t2.assignedbuilding = t1.assignedbuilding then 1 else 0 end as Mngr_Colo, t2.AssignedBuilding_SwipedIn as Mngr_Swiped, 
Num_Teammate, Num_Teammate_Colo, Num_ColoTeammate_Swiped from assignedattendance as t1
LEFT JOIN
(select * from assignedattendance) as t2
on t1.date = t2.date and t1.ManagerHashedID = t2.EmployeeHashID
LEFT JOIN
(select EmployeeHashID, date, count(distinct Teammate_ID) as Num_Teammate, SUM(Teammate_Colo) as Num_Teammate_Colo, SUM(ColoTeammate_Swiped) as Num_ColoTeammate_Swiped from 
(select t1.EmployeeHashID, t1.date, t1.AssignedBuilding_SwipedIn, t1.assignedbuilding, t2.EmployeeHashID as Teammate_ID, 
CASE WHEN t1.assignedbuilding = t2.assignedbuilding then 1 else 0 end as Teammate_Colo, 
CASE WHEN t1.assignedbuilding = t2.assignedbuilding AND t2.AssignedBuilding_SwipedIn = 1 then 1 else 0 end as ColoTeammate_Swiped from assignedattendance as t1
LEFT JOIN
(select * from assignedattendance) as t2
on t1.date = t2.date and t1.ManagerHashedID = t2.ManagerHashedID and t1.EmployeeHashID <> t2.EmployeeHashID)
group by EmployeeHashID, date) as t3
on t1.date = t3.date and t1.EmployeeHashID = t3.EmployeeHashID
where t1.ManagerHashedID IS NOT NULL
'''
sdf = spark.sql(query)

#Restrict to co-located manager and at least one co-located teammate
sdf = sdf.filter(sdf.Mngr_Colo == 1)
sdf = sdf.filter(sdf.Num_Teammate_Colo >= 1)

sdf.createOrReplaceTempView("t_sdf")

## US Time Series of Conditional Probability for the Four Groups (Fig 1a)

In [ ]:
#US Time Series Transformation for Conditional Probability
query = '''
select date, count(distinct EmployeeHashID) as HC, 
SUM(AssignedBuilding_SwipedIn) as Onsite_HC, 
SUM(CASE WHEN Mngr_Swiped = 1 and Num_ColoTeammate_Swiped >= 1 then 1 ELSE 0 end) as MngrOnTeamOn,
SUM(CASE WHEN Mngr_Swiped = 1 and Num_ColoTeammate_Swiped >= 1 and AssignedBuilding_SwipedIn = 1 then 1 ELSE 0 end) as Onsite_MngrOnTeamOn,
SUM(CASE WHEN Mngr_Swiped = 1 and Num_ColoTeammate_Swiped = 0 then 1 ELSE 0 end) as MngrOnTeamOff,
SUM(CASE WHEN Mngr_Swiped = 1 and Num_ColoTeammate_Swiped = 0 and AssignedBuilding_SwipedIn = 1 then 1 ELSE 0 end) as Onsite_MngrOnTeamOff,
SUM(CASE WHEN Mngr_Swiped = 0 and Num_ColoTeammate_Swiped >= 1 then 1 ELSE 0 end) as MngrOffTeamOn,
SUM(CASE WHEN Mngr_Swiped = 0 and Num_ColoTeammate_Swiped >= 1 and AssignedBuilding_SwipedIn = 1 then 1 ELSE 0 end) as Onsite_MngrOffTeamOn,
SUM(CASE WHEN Mngr_Swiped = 0 and Num_ColoTeammate_Swiped = 0 then 1 ELSE 0 end) as MngrOffTeamOff,
SUM(CASE WHEN Mngr_Swiped = 0 and Num_ColoTeammate_Swiped = 0 and AssignedBuilding_SwipedIn = 1 then 1 ELSE 0 end) as Onsite_MngrOffTeamOff
from t_sdf where assignedbuilding_region = 'US HQ' and date >= '2021-12-28' and date <= '2022-05-27' 
and date not in ('2021-12-31', '2022-01-17', '2022-02-21')
group by date
ORDER BY date
'''
us_df = spark.sql(query)
us_df.createOrReplaceTempView("US")

See the 'US_Time_Series_Viz' script for plotting the time series of conditional attendance probability

## TWFE Modeling for US, India, and Ireland (Fig 1b)

In [ ]:
#Pandas Transformation/processing
def processing(sdf, region, holidays, open_date):
    date_holidays = {}
    date_holidays['US'] = ['2021-12-31', '2022-01-17', '2022-02-21']
    date_holidays['IN'] = ['2022-01-26', '2022-04-14', '2022-04-15']
    date_holidays['IE'] = ['2022-03-17', '2022-03-18', '2022-04-18', '2022-06-06']
    
    df = sdf.filter(sdf.assignedbuilding_region == region).toPandas()
    df = df.loc[~df.date.isin(date_holidays[holidays])]

    df['date_dt'] = pd.to_datetime(df['date'])
    df['day_of_week'] = df['date_dt'].dt.strftime('%A')
    df = df.loc[~df.day_of_week.isin(['Saturday', 'Sunday'])]

    df['Perc_ColoTeammate_Swiped'] = df['Num_ColoTeammate_Swiped'] / df['Num_Teammate_Colo']

    #cohort filtering step 1: Present in 10 days before pre starts and 10 days after post ends
    open_date_dt = datetime.strptime(open_date, "%Y-%m-%d")
    pre_empid = list(df.loc[(df.date < open_date_dt - timedelta(days=60)) & (df.date >= open_date_dt - timedelta(days=70))].EmployeeHashID.unique())
    post_empid = list(df.loc[(df.date >= open_date_dt + timedelta(days=90)) & (df.date < open_date_dt + timedelta(days=100))].EmployeeHashID.unique())
    empid = [i for i in pre_empid if i in post_empid]

    df = df.loc[df.EmployeeHashID.isin(empid)]

    #cohort filter step 2: Sufficient records in pre and post
    temp1 = df.loc[(df.date >= open_date_dt - timedelta(days=60)) & (df.date < open_date_dt)].groupby('EmployeeHashID').count()['date'].reset_index()
    pre_empid2 = list(temp1.loc[temp1.date >= 15].EmployeeHashID.unique())
    temp2 = df.loc[(df.date >= open_date_dt + timedelta(days=30)) & (df.date <= open_date_dt + timedelta(days=90))].groupby('EmployeeHashID').count()['date'].reset_index()
    post_empid2 = list(temp2.loc[temp2.date >= 15].EmployeeHashID.unique())
    empid2 = [i for i in pre_empid2 if i in post_empid2]

    df = df.loc[df.EmployeeHashID.isin(empid2)]

    pre = df.loc[(df.date >= open_date_dt - timedelta(days=60)) & (df.date < open_date_dt)]
    post = df.loc[(df.date >= open_date_dt + timedelta(days=30)) & (df.date <= open_date_dt + timedelta(days=90))]

    pre_sdf = spark.createDataFrame(pre)
    pre_sdf.createOrReplaceTempView("pre")
    post_sdf = spark.createDataFrame(post)
    post_sdf.createOrReplaceTempView("post")

In [ ]:
%%sparkr
#function that returns bife coefficients 
bife_mod <- function(temp, New_Hire_ind = FALSE) {
    library(bife)
    temp$date <- as.factor(temp$date)
    temp$EmployeeHashID <- as.factor(temp$EmployeeHashID)
    
    if (New_Hire_ind == FALSE) {
        mod <- bias_corr(bife(AssignedBuilding_SwipedIn ~ Mngr_Swiped + Perc_ColoTeammate_Swiped + date | EmployeeHashID, data=temp))
        coef <- get_APEs(mod)
        print(paste('Manager coef: ', as.double(coef$delta[1])))
        print(paste('Teammate coef: ', as.double(coef$delta[2])))
    } else {
        mod <- bias_corr(bife(AssignedBuilding_SwipedIn ~ Mngr_Swiped + Perc_ColoTeammate_Swiped + HirePost20200301:Mngr_Swiped + HirePost20200301:Perc_ColoTeammate_Swiped + date | EmployeeHashID, data=temp))
        coef <- get_APEs(mod)
        print(paste('Manager coef: ', as.double(coef$delta[1])))
        print(paste('Teammate coef: ', as.double(coef$delta[2])))
        print(paste('Manager:New Hire coef: ', as.double(coef$delta[length(coef$delta)-1])))
        print(paste('Teammate:New Hire coef: ', as.double(coef$delta[length(coef$delta)])))
    }
    
}

#function that returns bootstrapped standard errors
bootstrap_se <- function(temp, New_Hire_ind = FALSE, replicates = 500) {
    require(bife)
    require(digest)
    require(dplyr)

    ManagerSwiped_mfx <- list()
    Perc_Colo_CoRep_Swiped_mfx <- list()
    newhire_mngr <- list()
    newhire_team <- list()
            
    for (i in 1:replicates) {
    temp$seed <- i
    temp$original <- paste0(temp$Mngr_ID, temp$seed)
    temp$hashed_team <- lapply(temp$original, function(original) digest(original, algo = "md5", serialize = FALSE))
    temp$rand <- substr(temp$hashed_team, start=3, stop=3) < "8"

    if (New_Hire_ind == TRUE) {
        est <- bias_corr(bife(AssignedBuilding_SwipedIn ~ Mngr_Swiped + Perc_ColoTeammate_Swiped + HirePost20200301:Mngr_Swiped + HirePost20200301:Perc_ColoTeammate_Swiped + date| EmployeeHashID, temp[which(temp$rand == TRUE), ]))
    } else {
        est <- bias_corr(bife(AssignedBuilding_SwipedIn ~ Mngr_Swiped + Perc_ColoTeammate_Swiped + date| EmployeeHashID, temp[which(temp$rand == TRUE), ]))
    }
    
    coefs <- get_APEs(est)
    ManagerSwiped_mfx[[i]] <- as.double(coefs$delta[1])
    Perc_Colo_CoRep_Swiped_mfx[[i]] <- as.double(coefs$delta[2])

    if (New_Hire_ind == TRUE) {
        newhire_mngr[[i]] <- as.double(coefs$delta[length(coefs$delta)-1])
        newhire_team[[i]] <- as.double(coefs$delta[length(coefs$delta)])
    }
    }

    if (New_Hire_ind == FALSE) {
        res = list("ManagerSwiped_mfx" = ManagerSwiped_mfx, "Perc_Colo_CoRep_Swiped_mfx" = Perc_Colo_CoRep_Swiped_mfx)
        print(paste("standard error for ManagerSwiped: ", sd(unlist(res$ManagerSwiped_mfx))))
        print(paste("standard error for Perc_Colo_CoRep_Swiped: ", sd(unlist(res$Perc_Colo_CoRep_Swiped_mfx))))
    } else {
        res = list("ManagerSwiped_mfx" = ManagerSwiped_mfx, "Perc_Colo_CoRep_Swiped_mfx" = Perc_Colo_CoRep_Swiped_mfx, newhire_mngr = "newhire_mngr", newhire_team = "newhire_team")
        print(paste("standard error for ManagerSwiped: ", sd(unlist(res$ManagerSwiped_mfx))))
        print(paste("standard error for Perc_Colo_CoRep_Swiped: ", sd(unlist(res$Perc_Colo_CoRep_Swiped_mfx))))
        print(paste("standard error for ManagerSwiped:New Hire: ", sd(unlist(res$newhire_mngr))))
        print(paste("standard error for Perc_Colo_CoRep_Swiped:New Hire: ", sd(unlist(res$newhire_team))))
    }
}

In [ ]:
#US
processing(t_sdf, 'US HQ', 'US', '2022-02-28')

In [ ]:
%%sparkr
pre <- as.data.frame(sql("select * from pre"))
bife_mod(pre)
bootstrap_se(pre)
post <- as.data.frame(sql("select * from post"))
bife_mod(post)
bootstrap_se(post)

In [ ]:
#India
processing(t_sdf, 'India', 'IN', '2022-03-08')

In [ ]:
%%sparkr
pre <- as.data.frame(sql("select * from pre"))
bife_mod(pre)
bootstrap_se(pre)
post <- as.data.frame(sql("select * from post"))
bife_mod(post)

In [ ]:
#Ireland
processing(t_sdf, 'India', 'IN', '2022-04-04')

In [ ]:
%%sparkr
pre <- as.data.frame(sql("select * from pre"))
bife_mod(pre)
bootstrap_se(pre)
post <- as.data.frame(sql("select * from post"))
bife_mod(post)

See the 'Paremetric_Estimation_Relative_Effects_Viz' script for plotting parametric estimations for co-attendance patterns across US Headquarters, India, and Ireland.

## Co-attendance Patterns by Role, New Hire Status, and Workspace Type (Fig 2)

In [ ]:
processing(t_sdf, 'US HQ', 'US', '2022-02-28')

In [ ]:
%%sparkr
# By Job Role
ER <- as.data.frame(sql("select * from post where JobCategory = 'Engineering & Research'"))
PM <- as.data.frame(sql("select * from post where JobCategory = 'Product & Program Management'"))
CO <- as.data.frame(sql("select * from post where JobCategory = 'Corp & Ops'"))

bife_mod(ER)
bootstrap_se(ER)
bife_mod(PM)
bootstrap_se(PM)
bife_mod(CO)
bootstrap_se(CO)

In [ ]:
%%sparkr
# By New Hire
ER <- as.data.frame(sql("select * from post where JobCategory = 'Engineering & Research'"))

bife_mod(ER, New_Hire_ind = TRUE)
bootstrap_se(ER, New_Hire_ind = TRUE)

In [ ]:
%%sparkr
# By Workspace Type
closed <- as.data.frame(sql("select * from post where JobCategory = 'Engineering & Research' and RoomTypeDesc = 'Office'"))
open <- as.data.frame(sql("select * from post where JobCategory = 'Engineering & Research' and RoomTypeDesc = 'Neighborhood'"))

bife_mod(closed)
bootstrap_se(closed)
bife_mod(open)
bootstrap_se(open)

See the 'Paremetric_Estimation_Relative_Effects_Viz' script for plotting TWFE relative effects comparsion by job role, new-hire status, and workspace type.